In [1]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-10-15 23:36:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-10-15 23:36:52 (1.26 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("jewelry_reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz" 

spark.sparkContext.addFile(url)
jewelry_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

jewelry_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
jewelry_data_df.count()

1767753

In [6]:
jewelry_data_df = jewelry_data_df.dropna(how='any')
jewelry_data_df = jewelry_data_df.dropDuplicates()
jewelry_data_df.count()

1767394

In [7]:
jewelry_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
from pyspark.sql.functions import col
from pyspark.sql.types import DateType, IntegerType
jewelry_data_df = jewelry_data_df.withColumn("review_date",col("review_date").cast(DateType()))
jewelry_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
review_id_table = jewelry_data_df["review_id", "customer_id", "product_id", "product_parent", "review_date"]
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000SHT04I05Z|   10989386|B00BC9ZCFQ|     412268594| 2013-08-04|
|R100L48N7NQJOM|   18778022|B0098Z105W|     498518502| 2014-09-29|
|R100W5CE099235|   23281221|B00G4EKH86|     258430791| 2014-05-23|
|R101L2Z28VA8ZM|   21148835|B00O96VYY2|     736039112| 2014-12-25|
|R101NZ4EWORURU|   45435568|B00C1G4LME|     345456390| 2013-09-26|
|R103I4TR9XD1VU|   15934715|B0040UMTDW|     913517322| 2012-01-01|
|R10451EZEGTTJ4|   18019354|B008SOTSEY|     707188081| 2014-10-23|
|R10454XRBFGGLU|   23200310|B003XYMXLY|     665660556| 2015-02-19|
|R1046FPO0U5JQZ|    6629323|B00VKNYBR4|     614133040| 2015-07-23|
|R104Q26FWKPR4N|   39979029|B003XT5ZZK|     290581820| 2014-05-02|
|R104SFUWPZGNKA|   31462035|B009R87SQU|     235825103| 2013-02-08|
|R1058QWV77ZW6R|   12344334|B00GOZUIZC|     713409458| 2014-05

In [10]:
products = jewelry_data_df["product_id", "product_title"]
products = products.dropDuplicates()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00LHJMHVS|FAIRY COUPLE Wome...|
|B00TPWQ6I4|Chic Alert Medica...|
|B00P837QR4|The Original Roll...|
|B00L280CN4|10pcs 18" 18 Inch...|
|B00KGU5T4Q|KONOV Mens Stainl...|
|B00JHF388K|Sterling Silver w...|
|B00FB05CES|CHOOSE YOUR COLOR...|
|B00D4KUMQ0|INBLUE Men's Stai...|
|B00GTCD2X0|Beautiful Carved ...|
|B00QUHZQP6|Authentic Chamili...|
|B00NJ317T8|Smitco LLC Silver...|
|B00658M8EC|925 Sterling Silv...|
|B00L43PV2Y|Hydia Steel Jewel...|
|B00U24CSP4|Platinum-Plated S...|
|B006OHTHZ2|316L Stainless St...|
|B00QW72KDA|PiercingJ Pair 12...|
|B006VN35L6|5mm Stainless Ste...|
|B00CGID7PY|925 Sterling Silv...|
|B00TXHSXWI|Silver Forest Kno...|
|B004Z1EQRS|PammyJ Silvertone...|
+----------+--------------------+
only showing top 20 rows



In [11]:
customers = jewelry_data_df.groupBy("customer_id").count()
customers = customers.withColumn("count",col("count").cast(IntegerType()))
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   14542376|            11|
|     195263|             2|
|     726270|             1|
|   43776283|             1|
|    2074030|             8|
|   18503260|             2|
|   46017005|             1|
|   52807885|             1|
|   47816064|             1|
|   43156541|             1|
|   16200102|             6|
|   20665363|             1|
|   18194793|             1|
|   21440635|             2|
|    3065770|             1|
|   37392783|             5|
|   13627727|             1|
|   16260707|            14|
|   37398268|             1|
|   27716699|             3|
+-----------+--------------+
only showing top 20 rows



In [12]:
vine_table = jewelry_data_df["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]
vine_table = vine_table.withColumn("star_rating",col("star_rating").cast(IntegerType()))
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000SHT04I05Z|          4|            1|          1|   N|
|R100L48N7NQJOM|          5|            0|          0|   N|
|R100W5CE099235|          1|            3|          3|   N|
|R101L2Z28VA8ZM|          5|            0|          0|   N|
|R101NZ4EWORURU|          5|            0|          0|   N|
|R103I4TR9XD1VU|          4|            0|          1|   N|
|R10451EZEGTTJ4|          5|            0|          0|   N|
|R10454XRBFGGLU|          4|            0|          0|   N|
|R1046FPO0U5JQZ|          4|            1|          1|   N|
|R104Q26FWKPR4N|          5|            2|          3|   N|
|R104SFUWPZGNKA|          5|            0|          0|   N|
|R1058QWV77ZW6R|          3|            1|          1|   N|
|R105CYSX0JNXQR|          1|            1|          1|   N|
|R106A76VCQU3TY|          5|            

In [15]:
mode = "append"
jdbc_url="jdbc:postgresql://<insert link to RDS here>:5432/<insert database name here>"
config = {"user":"root", 
          "password": "<insert password here>", 
          "driver":"org.postgresql.Driver"}

In [16]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)